# Learning from Imbalanced Data

과제내용
- Imbalanced learning 기법 간 성능 비교
- 상세내용은 아래 실험방법 참조


•데이터
- Kaggle Competition으로 2017년말에 개최되었던 "WSDM - KKBox's Music Recommendation Challenge" 데이터에 Feature Engineering을 적용한 것을 사용한다. 
(첨부화일)


•실험방법
- class 변수인 'target' 필드에서 1의 값이 전체 데이터의 0.1%, 1%, 10%가 되도록 축소하여 3가지의 data imbalance 상황을 만든다.
(필요시 위에 더하여 새로운 imbalance 상황을 추가할 수 있음)
- 각각의 data imbalance 상황에서, 여러 Imbalanced learning 기법을 적용한 데이터에 대해  RandomForest 알고리즘을 적용하여 성능을 평가하고 비교한다. 
(존재하는 feature를 수정하지 않고 그대로 사용함)
- 평가척도로는 Accuracy, ROC-AUC, Recall, Precision, F1-score 등 5가지를 사용하고, "Survey of resampling techniques for improving classification performance in unbalanced datasets" 논문(첨부화일)에서 테스트한 기법 중 imbalanced-learn 패키지에서 지원하는 것은 모두 테스트한다.


•제출물
- 소스코드(화일명: 이름1-이름2.ipynb)와 결과보고서(화일명: 이름1-이름2.ppt)
- 4월22일 수업시간에 팀당 10분 이내 발표


•유의사항
- 마감시간 내에 제출한 과제만 평가함. 
- 반드시, 담당교수 이메일(www4u@kookmin.ac.kr)로 제출해야 하고, 메일 제목은 "팀과제(2차) 이름1-이름2"로 할 것.


In [21]:
# ! pip install -U imbalanced-learn # U 하면 새버전으로 깔림.

In [22]:
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from collections import Counter
from sklearn.svm import SVC

In [23]:
data = pd.read_csv("WSDM_train_full_features.csv")

In [24]:
data.head()

,target,user_idx,song_idx,ssn_idx,sst_idx,st_idx,song_length,genr_idx,art_idx,com_idx,...,lyricists_count,composer_count,is_featured,artist_count,artist_composer,artist_composer_lyricist,song_lang_boolean,smaller_song,count_song_played,count_artist_played
0,1,0,0,0,0,0,206471,7,1877,7450,...,0,2,0,0,0,0,0,1,215,1140
1,1,1,1,1,1,1,284584,18,92,2,...,0,1,0,0,0,0,0,0,1,303616
2,1,1,2,1,1,1,225396,18,527,100234,...,0,1,0,0,0,0,0,1,4,289
3,1,1,3,1,1,1,255512,229,89592,193180,...,0,1,0,0,0,0,0,0,1,1
4,1,0,4,0,0,0,187802,13,195,137,...,0,3,0,0,0,0,0,1,412,427


In [25]:
data=data.iloc[0:round(len(data)/2),:]
data.shape

(3688709, 30)

In [26]:
data=data.iloc[0:round(len(data)/2),:]
data.shape

(1844354, 30)

In [27]:
data=data.iloc[0:round(len(data)/2),:]
data.shape

(922177, 30)

In [28]:
data_0 = data[data['target']==0]
data_1 = data[data['target']==1]

In [29]:
print(data_0.shape[0])
print(data_1.shape[0])  # 비슷한 크기 -> 둘 중 하나의 값을 줄여야한다.

279715
642462


In [30]:
data_1_01 = data_1.iloc[np.random.randint(0,len(data_1),size=round((1/9)*len(data_0)))]
print(data_1_01.shape)
print(data_1_01.head())

(31079, 30)
        target  user_idx  song_idx  ssn_idx  sst_idx  st_idx  song_length  \
876981       1      8046     16941        2        4       5       263627   
708337       1     12629       163        1        1       1       189361   
721717       1      7232       459        1        1       2       266448   
596165       1      8460      6498        1        1       2       196440   
64625        1      5011        46        1        1       2       215144   

        genr_idx  art_idx  com_idx         ...           lyricists_count  \
876981        26     2906    29791         ...                         0   
708337        97     1146    50766         ...                         6   
721717         4     1194    44391         ...                         0   
596165         1       79    35768         ...                         0   
64625          4       58       48         ...                         0   

        composer_count  is_featured  artist_count  artist_composer  

In [31]:
df = pd.concat([data_0,data_1_01],axis=0)
df.head()
df.shape

(310794, 30)

In [32]:
data.shape # 원본데이터

(922177, 30)

In [33]:
X = df.iloc[:,1:30]
y = df.iloc[:,0]

In [34]:
# 학습/평가 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape

((233095, 29), (77699, 29))

## SCORE 모아두기

In [107]:
accuracy = []
roc_auc = []
f1 = []
recall = []
precision = []

In [104]:
# 원래 데이터를 사용했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_train, y_train).predict(X_test)
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.9002432463738272 
 roc_auc_score : 0.9999999999999999 
 f1_score : 0.13772388474802535 
 recall_score : 0.08083050404805432 
 precision_score : 0.4650638617580766


In [108]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

In [110]:
print(accuracy)
print(roc_auc )
print(f1 )
print(recall)
print(precision )

[0.9002432463738272]
[0.9999999999999999]
[0.13772388474802535]
[0.08083050404805432]
[0.4650638617580766]


In [ ]:
plt.barplot()

# Under_Sampling
# 2-1. NearMiss1. 

In [35]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(version=1,random_state=0)

In [36]:
X_resampled_nm1, y_resampled = nm1.fit_sample(X_train, y_train)
X_resampled_nm1.shape

(46842, 29)

In [43]:
accuracy_score(y_test, y_pred)

0.6991724475218472

In [44]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled_nm1, y_resampled).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_resampled_nm1, y_resampled).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.2677898042445849 
 roc_auc_score : 0.4844836143411864 
 f1_score : 0.16880460509014406 
 recall_score : 0.7543745103160094 
 precision_score : 0.09503520431664145


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 2-2. NearMiss2. 

from imblearn.under_sampling import NearMiss
nm2 = NearMiss(version=2,random_state=0)

X_resampled_nm2, y_resampled = nm2.fit_sample(X_train, y_train)
X_resampled_nm2.shape

# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled_nm2, y_resampled).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 2-3. NearMiss3. 

In [45]:
from imblearn.under_sampling import NearMiss
nm3 = NearMiss(version=3,random_state=0)

In [46]:
X_resampled_nm3, y_resampled = nm3.fit_sample(X_train, y_train)
X_resampled_nm3.shape

(46842, 29)

In [47]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled_nm3, y_resampled).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.6074724256425437 
 roc_auc_score : 0.7246729883305413 
 f1_score : 0.24717991755732727 
 recall_score : 0.6538260642465396 
 precision_score : 0.15239689544970325


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 3. RandomUnderSampler

In [39]:
from imblearn.under_sampling import RandomUnderSampler 

In [40]:
rus = RandomUnderSampler(random_state=0)

In [41]:
X_res, y_res = rus.fit_resample(X_train, y_train)

In [48]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n roc_auc_score :',roc_auc_score(y_test, y_proba), '\n f1_score :', f1_score(y_test, y_pred), '\n recall_score :',recall_score(y_test, y_pred),'\n precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.6991724475218472 
 roc_auc_score : 0.7246729883305413 
 f1_score : 0.28876582278481017 
 recall_score : 0.6196134761034212 
 precision_score : 0.18824882964373563


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 4. Condensed Nearest Neighbor (CNN)

from imblearn.under_sampling import CondensedNearestNeighbour

cnn = CondensedNearestNeighbour(random_state=0)

X_res, y_res = cnn.fit_resample(X_train, y_train)

# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred)

accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 5. Edited Nearest Neighbor (ENN)

In [49]:
from imblearn.under_sampling import EditedNearestNeighbours

In [50]:
enn = EditedNearestNeighbours(random_state=0)

In [51]:
X_res, y_res = enn.fit_resample(X_train, y_train)

In [52]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n roc_auc_score :',roc_auc_score(y_test, y_proba), '\n f1_score :', f1_score(y_test, y_pred), '\n recall_score :',recall_score(y_test, y_pred), '\n precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8888788787500482 
 roc_auc_score : 0.7162073874881381 
 f1_score : 0.2165154264972777 
 recall_score : 0.15578480020893185 
 precision_score : 0.3548483045806068


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 6. Repeated Edited Nearest Neighbor 

In [53]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours 

In [54]:
renn = RepeatedEditedNearestNeighbours(random_state=0)

In [55]:
X_res, y_res = renn.fit_resample(X_train, y_train)

In [56]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8679905790293312 
 roc_auc_score : 0.7198761001414576 
 f1_score : 0.25343911492830623 
 recall_score : 0.22734395403499608 
 precision_score : 0.2863015951323795


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 7. Tomek Link Removal

In [57]:
from imblearn.under_sampling import TomekLinks 

In [58]:
tl = TomekLinks(random_state=0)

In [59]:
X_res, y_res = tl.fit_resample(X_train, y_train)

In [60]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8984156810254958 
 roc_auc_score : 0.7060543613843996 
 f1_score : 0.15465352897076148 
 recall_score : 0.09428049098981457 
 precision_score : 0.43001786777843953


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# Oversampling methods
# 1. Random oversampling of minority class

In [61]:
from imblearn.over_sampling import RandomOverSampler

In [62]:
ros = RandomOverSampler(random_state=0)

In [63]:
X_res, y_res = ros.fit_resample(X_train, y_train)

In [64]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8933448306928017 
 roc_auc_score : 0.7195308214597987 
 f1_score : 0.1920639563225115 
 recall_score : 0.1286236615304257 
 precision_score : 0.37899191996921894


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 2. SMOTE

In [65]:
from imblearn.over_sampling import SMOTE

In [66]:
X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X_train, y_train)

In [67]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled, y_resampled).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8777976550534756 
 roc_auc_score : 0.7195308214597987 
 f1_score : 0.287643484132343 
 recall_score : 0.25032645599373204 
 precision_score : 0.3380356198201375


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 3-1. Borderline-SMOTE1

In [68]:
from imblearn.over_sampling import BorderlineSMOTE

In [69]:
X_resampled, y_resampled = BorderlineSMOTE(kind='borderline-1',random_state=0).fit_resample(X_train, y_train)

In [70]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled, y_resampled).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8815814875352321 
 roc_auc_score : 0.7195308214597987 
 f1_score : 0.27097694318992155 
 recall_score : 0.22329589971271874 
 precision_score : 0.3445496675397945


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 3-2.Borderline-SMOTE2

In [71]:
X_resampled, y_resampled = BorderlineSMOTE(kind='borderline-2',random_state=0).fit_resample(X_train, y_train)

In [72]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_resampled, y_resampled).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8772571075560818 
 roc_auc_score : 0.7195308214597987 
 f1_score : 0.2664410429966926 
 recall_score : 0.22616871245756073 
 precision_score : 0.3241624555493169


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# Combination methods
# 1. SMOTE + Tomek Link Removal

In [73]:
from imblearn.combine import SMOTETomek

In [74]:
smt = SMOTETomek(random_state=0)

In [75]:
 X_res, y_res = smt.fit_resample(X_train, y_train)

In [76]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8744642788195472 
 roc_auc_score : 0.7378991761975447 
 f1_score : 0.2773744258408653 
 recall_score : 0.24445024810655525 
 precision_score : 0.32054794520547947


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 2. SMOTE + ENN

In [77]:
 from imblearn.combine import SMOTEENN

In [78]:
sme = SMOTEENN(random_state=42)

In [79]:
X_res, y_res = sme.fit_resample(X_train, y_train)

In [80]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


 accuracy_score : 0.8470250582375578 
 roc_auc_score : 0.7402385234654318 
 f1_score : 0.3024647887323944 
 recall_score : 0.3365108383389919 
 precision_score : 0.2746749094009806


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# Ensemble Methods
# 1. EasyEnsemble

In [81]:
from imblearn.ensemble import EasyEnsemble

In [82]:
ee = EasyEnsemble(random_state=0)

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class EasyEnsemble is deprecated; EasyEnsemble is deprecated in 0.4 and will be removed in 0.6. Use EasyEnsembleClassifier instead.
  warnings.warn(msg, category=DeprecationWarning)


In [83]:
X_res, y_res = ee.fit_resample(X, y)

In [84]:
X_res=X_res.reshape(-1,29) # -1 행개수를 지정하지 않겠다.
y_res=y_res.reshape(-1,1) # -1 행개수를 지정하지 않겠다.

In [85]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports u

In [86]:
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n roc_auc_score :',roc_auc_score(y_test, y_proba), '\n f1_score :', f1_score(y_test, y_pred), '\n recall_score :',recall_score(y_test, y_pred),'\n precision_score :', precision_score(y_test, y_pred))

 accuracy_score : 0.9904117170105149 
 roc_auc_score : 0.9998999466748926 
 f1_score : 0.9536143453085113 
 recall_score : 1.0 
 precision_score : 0.9113411876710699


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

# 2. BalanceCascade

In [87]:
from imblearn.ensemble import BalanceCascade

In [88]:
bc = BalanceCascade(random_state=42)

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class BalanceCascade is deprecated; BalanceCascade is deprecated in 0.4 and will be removed in 0.6.
  warnings.warn(msg, category=DeprecationWarning)


In [89]:
X_res, y_res = bc.fit_resample(X, y)

In [90]:
X_res=X_res.reshape(-1,29) # -1 행개수를 지정하지 않겠다.
y_res=y_res.reshape(-1,1) # -1 행개수를 지정하지 않겠다.

In [91]:
# 합성 데이터를 추가했을 때의 모델성능
y_pred = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict(X_test)
y_proba = RandomForestClassifier(random_state=0).fit(X_res, y_res).predict_proba(X_test)[:,1]
print(' accuracy_score :', accuracy_score(y_test, y_pred), '\n' ,'roc_auc_score :',roc_auc_score(y_test, y_proba), '\n','f1_score :', f1_score(y_test, y_pred), '\n','recall_score :',recall_score(y_test, y_pred),'\n','precision_score :', precision_score(y_test, y_pred))

C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\Bae yuna\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Bae yuna\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports u

 accuracy_score : 0.9998841683934157 
 roc_auc_score : 0.9999999999999999 
 f1_score : 0.9994127243066884 
 recall_score : 1.0 
 precision_score : 0.9988261379940002


In [ ]:
accuracy.append(accuracy_score(y_test, y_pred))
roc_auc.append(roc_auc_score(y_test, y_proba))
f1.append(f1_score(y_test, y_pred))
recall.append(recall_score(y_test, y_pred))
precision.append(precision_score(y_test, y_pred))

In [ ]:
print(accuracy)
print(roc_auc )
print(f1 )
print(recall)
print(precision )